In [15]:
# Load dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from lifelines import KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts

# Set pandas option to display all columns
pd.set_option('display.max_columns', None)

In [27]:
pro1 = pd.read_csv('/Users/Apple/Downloads/All_processed_data/ALS_Proact_Data_after_EDA_clinical_clustering_and_trial_groups.csv')
pro1.head(3)

,subject_id,Event,Age,Sex,Height,Weight,Onset_site,Riluzole_use,Study_Arm,European,Disease_Duration,Stage,Trial_Onset,ALSFRS_RT,ALSFRS_Rasch,Diagnosis_delay,Predicted_FVC,Actual_FVC,Vital_Capacity,slope,TRICALS,TRICALS_eligible,TRICALS_risk_cluster,AgeCluster,DataCluster,Age.,Height.,Weight.,ALSFRS_RT.,ALSFRS_Rasch.,Diagnosis_delay.,Predicted_FVC.,Actual_FVC.,Trial_Onset.,TRICALS.,rough_group,rough_group2,gmm_group,gmm_group2,latent_group,trial_group
0,121,1,52.00,Female,173.000,74.600,limb,Yes,Active,1.0,31.849540,4,17.557162,27.0,21.1,10.607096,5.108100,3.22,63.037137,-0.659350,-4.184806,eligible,0.0,1,0,-10.00,7.900,9.769,4.0,1.9,-1.003942,0.600200,0.183964,6.701708,-0.449482,Medium,2.0,0,2,0,Trial_18mo
1,226,1,72.00,Male,174.000,71.500,bulbar,Yes,Active,1.0,29.113666,4,13.910644,26.0,20.6,11.320302,4.570600,2.57,56.228941,-0.755659,-3.279029,eligible,0.0,0,1,10.00,8.900,6.669,3.0,1.4,-0.290736,0.062700,-0.466036,3.055191,0.456295,Medium,2.0,0,2,0,Trial_18mo
2,624,1,65.14,Female,164.073,64.831,limb,Yes,Active,1.0,28.748357,4,11.643890,23.0,19.2,14.095926,4.309306,1.63,37.825112,-0.869615,-3.282845,eligible,0.0,0,0,3.14,-1.027,0.000,0.0,0.0,2.484888,-0.198594,-1.406036,0.788436,0.452479,Medium,2.0,0,2,0,Trial_12mo


In [17]:
# pro1.columns

In [ ]:
pro = pro1[['subject_id', 'Event', 'Age', 'Sex', 'Onset_site','Riluzole_use', 'Study_Arm', 'European',
             'Trial_Onset', 'ALSFRS_Rasch', 'Diagnosis_delay','Vital_Capacity', 'TRICALS', 'trial_group']]

In [19]:
lica1 = pd.read_csv('/Users/Apple/Downloads/All_processed_data/LICALS_Data_EDA.csv')
lica1.head(3)

,subject_id,Event,Vital_capacity,El_Escorial,Sex,Height,Onset_site,Study_Arm,European,diag_to_death,Age_Rand,Lithum_use,Diagnostic_Delay,Study_Duration,Stage,ALSFRS_RT,ALSFRS_Rasch,onset_site_cat,ftd,dxdelay,slope,tAge,tdxdelay,minus_slope,tslope,tfvc,TRICALS,Age.,Height.,ALSFRS_RT.,ALSFRS_Rasch.,Diagnostic_Delay.,TRICALS.
0,P01001,0,107.0,0,Male,171.0,Limb,Placebo,1,25.512484,54.762491,Yes,13.600526,19.000000,4,35.0,25.7,0,0,14.600526,-0.684211,3.334522,1.206063,0.684211,1.129233,1.901316,-4.921357,-4.506502,-2.0,-5.0,-3.5,-2.496715,0.133766
1,P01002,1,99.0,0,Female,164.0,Limb,Placebo,1,29.434954,61.160849,Yes,20.137976,21.400788,4,43.0,32.2,0,0,21.137976,-0.233636,2.673333,1.436296,0.233636,1.731264,2.015139,-6.347018,1.891855,-9.0,3.0,3.0,4.040736,-1.291896
2,P01003,0,102.0,0,Male,182.0,Limb,Active,1,23.049606,46.702259,Yes,15.571616,19.000000,4,43.0,32.2,0,0,16.571616,-0.263158,4.584840,1.281921,0.263158,1.659404,1.970540,-6.373852,-12.566735,9.0,3.0,3.0,-0.525624,-1.318729


In [20]:
# lica1.columns

In [ ]:
lica = lica1[['subject_id', 'Event', 'Age_Rand', 'Sex', 'Onset_site','Lithum_use', 'Study_Arm', 'European', 
               'Study_Duration', 'ALSFRS_Rasch','Diagnostic_Delay', 'Vital_capacity',  'TRICALS']]

In [28]:
miro1 = pd.read_csv('/Users/Apple/Downloads/All_processed_data/MIROCALS_Data_EDA.csv')
miro1.head(3)

,ID,C9,SEX,AGEONSET_C,AGEDIAG_C,AGEINC_C,AGERANDO_C,RECLIFE,STATUS,INC_SITE_ONSET,SVCHP_INC,INC_DIAG,ARM_C,ALSFRS_RND,AGE_C,1. Speech_RD,2. Salivation_RD,3. Swallowing_RD,4. Handwriting_RD,5a. Cutting without gastrostomy_RD,6. Dressing and hygiene_RD,7. Turning in bed and adjusting bed clothes_RD,PNFHCSF_RD_3GTRT,8. Walking_RD,9. Climbing stairs_RD,10. Dyspnea_RD,11. Orthopnea_RD,12. Respiratory insufficiency_RD,End_diag_El_Escor,Stage_rnd,ALSFRS_Rasch,Onset_to_Rand,Study_Duration,Disease_Duration,Diagnostic_delay,onset_site_cat,ftd,dxdelay,slope,tAge,tdxdelay,minus_slope,tslope,tfvc,El_Escorial,TRICALS,Age.,ALSFRS_RND.,ALSFRS_Rasch.,Diagnostic_Delay.,TRICALS.
0,1-62478,0.0,F,69.391,70.650,70.927,70.927,327.0,1.0,Limb,99.0,Clinically probable ALS - laboratory supported,Placebo,39.0,70.93,4.0,4.0,4.0,4.0,4.0,3.0,3.0,low,2.0,0.0,3.0,4.0,4.0,Clinically probable ALS,3.0,28.4,18.432,11.742444,29.174444,15.108,0,0,16.108,-0.766450,1.987819,1.264646,0.766450,1.074306,2.015139,0,-4.550056,11.217,-1.0,-0.8,6.222,0.129100
1,1-62479,1.0,F,63.754,65.054,65.391,65.391,137.0,1.0,Bulbar,81.0,Clinically definite ALS,IL2,25.0,65.39,2.0,2.0,2.0,3.0,2.0,2.0,1.0,high,2.0,0.0,3.0,2.0,4.0,Clinically definite ALS,3.0,20.1,19.644,5.500657,24.144657,15.600,1,0,16.600,-4.181319,2.338644,1.282968,4.181319,0.483294,2.345679,1,-2.933660,5.681,-15.0,-9.1,6.714,1.745496
2,1-62480,1.0,M,56.167,56.909,57.024,57.024,640.0,0.0,Limb,96.0,Clinically probable ALS,IL2,42.0,57.02,4.0,4.0,4.0,3.0,3.0,3.0,4.0,low,3.0,2.0,4.0,4.0,4.0,Clinically probable ALS,3.0,31.1,10.284,22.024967,31.308967,8.904,0,0,9.904,-0.272418,3.075280,0.995188,0.272418,1.638644,2.062287,0,-5.212384,-2.686,2.0,1.9,0.018,-0.533228


In [25]:
# miro1.columns

In [ ]:
miro = miro1[['ID', 'STATUS', 'AGEONSET_C', 'SEX', 'INC_SITE_ONSET', 'PNFHCSF_RD_3GTRT', 'ARM_C', #'Race'
              'Study_Duration', 'ALSFRS_Rasch', 'Diagnostic_delay','SVCHP_INC', 'TRICALS']]